In [1]:
import sys
from pathlib import Path

# Assume the notebook lives in PROJECT_ROOT / "notebooks"
project_root = Path.cwd().parent
print("Detected project root:", project_root)

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print("Added to sys.path.")

# Now just import from the package – no manual path overrides
from vector_pipeline.settings import DATA_PATH, CHROMA_DIR, COMBINED_TEXT_COLUMN
from vector_pipeline.ingestion import build_or_load_vectorstore
from vector_pipeline.retrieval import (
    preview_vectorstore,
    debug_single_retrieval,
    debug_single_rag,
    interactive_retrieval_chat,
    interactive_rag_chat,
)

print("DATA_PATH:", DATA_PATH.resolve(), "-> exists:", DATA_PATH.exists())
print("CHROMA_DIR:", CHROMA_DIR.resolve())


Detected project root: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data
Added to sys.path.


`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


DATA_PATH: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/data/processed_product.pkl -> exists: True
CHROMA_DIR: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/chroma_db


In [2]:
import pandas as pd

df = pd.read_pickle(DATA_PATH)

print("Data path:", DATA_PATH)
print("Columns:", list(df.columns))
print("Number of rows:", len(df))

df["combined_length"] = (
    df[COMBINED_TEXT_COLUMN].fillna("").astype(str).str.len()
)

df[[COMBINED_TEXT_COLUMN, "combined_length"]].head()


Data path: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/data/processed_product.pkl
Columns: ['rating', 'rating_count', 'price', 'store', 'product_id', 'color', 'combined_text']
Number of rows: 3734


,combined_text,combined_length
0,Product Title: Ernie Ball Mondo Slinky Nickelw...,1532
1,Product Title: BONAOK Wireless Bluetooth Karao...,1515
2,Product Title: Boya BYM1 by Shotgun Video Micr...,1506
3,Product Title: D'Addario Guitar Strings - Phos...,1792
4,Product Title: Amazon Basics Adjustable Guitar...,820


In [3]:
vectorstore = build_or_load_vectorstore()

print("Vectorstore ready ✅")
print("Persist directory:", CHROMA_DIR)
print("Collection name:", vectorstore._collection.name)


Loading existing Chroma DB from /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/chroma_db ...
Vectorstore loaded successfully.
Document count: 6401
Vectorstore ready ✅
Persist directory: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/chroma_db
Collection name: langchain


In [7]:
preview_vectorstore(vectorstore, k=5)


Got 5 documents from vectorstore.

=== Doc 1 ===
Metadata:
  rating_count: 5399
  row_index: 269
  store: Donner
  start_index: 1827
  color: Dazzle pattern
  rating: 4.7
  product_id: B09XB69QQ4
  price: 7.19

Text snippet:
  Description:

----------------------------------------------------------------------

=== Doc 2 ===
Metadata:
  product_id: B07DD9KCX4
  store: TOSING
  price: 179.99
  rating: 4.1
  rating_count: 1686
  start_index: 1935
  color: Gold
  row_index: 871

Text snippet:
  Description:

----------------------------------------------------------------------

=== Doc 3 ===
Metadata:
  product_id: B0073J2VBC
  color: Not Specified
  price: 11.99
  rating: 4.7
  row_index: 1704
  rating_count: 1065
  start_index: 2275
  store: PWR+

Text snippet:
  Description:

----------------------------------------------------------------------

=== Doc 4 ===
Metadata:
  rating: 4.6
  color: Black
  product_id: B07TM115DZ
  price: 31.97
  store: Bim-Bam-Bom
  rating_count: 940
  star

In [9]:
MANUAL_QUERY = "wireless headphones with noise cancelling"
USE_RERANKER = True

debug_single_retrieval(
    vectorstore,
    query=MANUAL_QUERY,
    k=4,
    use_reranker=USE_RERANKER
)



Retrieved 4 documents for query: 'wireless headphones with noise cancelling'
Reranker enabled: True

──────────────────────────────────────────────
🛍️  Unknown product
   ID: B07TV7J574
   Brand: Alvoxcon
   Price: 49.99
   Description snippet:
   Features: SUPERIOR SOUND & PERFORMANCE – Dual uhf wireless lapel microphone delivers crystal clear sound & metal belt pack transmitter with mini receiver offer noise cancelling transmission range up to 40ft without feedback or distortion BEST CHOICE 
──────────────────────────────────────────────
──────────────────────────────────────────────
🛍️  Unknown product
   ID: B081Z8P7K2
   Brand: Alvoxcon
   Price: 49.99
   Description snippet:
   Features: SUPERIOR SOUND & PERFORMANCE – Dual uhf wireless lapel microphone delivers crystal clear sound & metal belt pack transmitter with mini receiver offer noise cancelling transmission range up to 40ft without feedback or distortion BEST CHOICE 
──────────────────────────────────────────────
────────

In [ ]:
RAG_QUERY = "What wireless headphones have the best battery life?"
USE_RERANKER = True

debug_single_rag(
    vectorstore,
    query=RAG_QUERY,
    k=4,
    use_reranker=USE_RERANKER,
)


RAG answer for query: 'What wireless headphones have the best battery life?'  (reranker=True)

Use ONLY the following product information to answer the question.
If the answer is not in the context, say you don't know.

CONTEXT:
Bluetooth for Wireless Music Streaming: Built-in Bluetooth for Wireless Music Streaming: Simple & Hassle-Free Pairing: Simple & Hassle-Free Pairing: Works with All of Today’s Latest Devices: Works with All of Today’s Latest Devices: Smartphones, Tablets, Laptops, Computers, etc.: Smartphones, Tablets, Laptops, Computers, etc.: Bluetooth Version: 4.0: Bluetooth Version: 4.0: Bluetooth Network Name: ‘PYLEUSA’: Bluetooth Network Name: ‘PYLEUSA’: Wireless Range: Up to 32’ ft.: Wireless Range: Up to 32’ ft.: What’s in the Box:: PA Speaker System: PA Speaker System: Remote Control: Remote Control: Power Adapter: Power Adapter: Technical Specs:: Power Output: 400 Watt MAX: Power Output: 400 Watt MAX: Sound System: 8’’ -inch Subwoofer: Sound System: 8’’ -inch Subwoofer

In [ ]:
USE_RERANKER = True
interactive_retrieval_chat(vectorstore, use_reranker=USE_RERANKER)


Semantic Product Search — type 'exit' to stop.

Reranker enabled: True


Bot: I found 3 related products:

──────────────────────────────────────────────
🛍️  Unknown product
   ID: B09XF54V38
   Brand: XIAOKOA
   Price: 29.88
   Description snippet:
   Features: 🎤Upgraded Head Microphone: wireless microphone headset with LED Display,it can be clearly displayed the frequency,volume and battery status.▶Up to 165 feet transmission range in an open areal.▶Support 40 sets to use at the same time.🔔Not Su
──────────────────────────────────────────────
──────────────────────────────────────────────
🛍️  Unknown product
   ID: B074C125TN
   Brand: FIFINE
   Price: 42.99
   Description snippet:
   Features: [Free Your Hands And Mind] USB wireless system brings the convenience of wireless to your communications and recordings. Ease of use, accurate response and lightweight construction allowing you to roam the room without having to hold a micr
──────────────────────────────────────────────
──────

In [ ]:
USE_RERANKER = True
interactive_rag_chat(vectorstore, use_reranker=USE_RERANKER)


RAG Chat — ask about any product! (type 'exit' to stop)

Reranker enabled: True

Bot: Goodbye 👋


In [ ]:
from vector_pipeline.retrieval import rag_answer

query = "what properties should I look at while choosing a good gaming computer"
answer = rag_answer(query, vs=vectorstore, k=4, use_reranker=True)
print(answer)

Use ONLY the following product information to answer the question.
If the answer is not in the context, say you don't know.

CONTEXT:
4 or Celeron-compatible CPU or faster (multicore CPU recommended); 1 GB RAM (2 GB recommended); Windows 10, Windows 8, Windows 7, Windows Vista, or Windows XP; Windows-compatible sound card (ASIO driver support recommended); QuickTime recommended. Available USB port.

---

Features: Next Generation Feature Packed Controller : Ultra Portable two deck DJ Controller for Serato DJ Intro (Included) for Mac and PC with high resolution 5 inch metal jog wheels Connectivity Covered : On board pro grade class compliant audio interface with 2 headphone outputs and main output for seamless connection to mixers & PA systems Mix Control : Two physical decks outfitted with a crossfader, dedicated browsing controls, individual filter controls, touch capacitive jog wheels and independent transport controls access to critical, must have tools for seamless blending are rig